# Análise Associação e Correlação

## Módulos

In [1]:
# instalar versão atualizada
! sudo pip install scipy==1.7.1

     |████████████████████████████████| 28.5 MB 1.3 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
# instalar nova ferramenta para testes de hipoteses
! sudo pip install pingouin

     |████████████████████████████████| 182 kB 5.1 MB/s 
     |████████████████████████████████| 9.8 MB 24.1 MB/s 
  Created wheel for pingouin: filename=pingouin-0.5.0-py3-none-any.whl size=193661 sha256=73389c8f1738cee98c221b133fee5f6a72fc38e820c91ea7e4ee053d9e1beecb
  Stored in directory: /root/.cache/pip/wheels/14/46/f9/cedd81d68d2515c24bbbd000d5b347e4fe092ccc4b568f7f70
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=363642c8daa18f03366337bde4c859232815f08fca3b665b4ee326109fda9cb2
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built pingouin littleutils
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [3]:
# gerais
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from scipy import stats as st
import pingouin as pg

## Dados

In [5]:
dados_tratamentos = pd.read_csv('dados_tratamentos.csv', sep = ';')
dados_tratamentos.head()

,id,tratamento,evolucao
0,1,tratado,melhorou
1,2,tratado,melhorou
2,3,nao-tratado,melhorou
3,4,tratado,melhorou
4,5,tratado,nao-melhorou


In [6]:
dados_nps = pd.read_csv('nps_example.csv', sep = ';')
dados_nps.head()

,id,response_status,how_long_listening,age,nps_score,gender
0,11706300,Complete,Less than 6 months,25-34,10.0,Female
1,11706302,Complete,1 year to less than 3 years,25-34,10.0,Female
2,11706307,Complete,6 months to less than a year,35-44,10.0,Female
3,11706312,Complete,Less than 6 months,35-44,10.0,Female
4,11706316,Complete,6 months to less than a year,25-34,10.0,Male


In [7]:
dados_bolsa = pd.read_csv('dados_bolsa.csv', sep = ';', decimal = ',')
dados_bolsa

,data,petr4,bbdc3,vale5,ambv4,itub4
0,2004-05-26,-0.002270,0.009524,0.013699,0.032668,0.009843
1,2004-05-27,0.026301,0.028396,0.028303,0.014060,0.021442
2,2004-05-28,-0.016484,-0.000092,-0.025261,0.019151,0.000000
3,2004-05-31,0.010986,-0.009174,0.011235,-0.001786,0.000000
4,2004-06-01,0.028142,0.023056,-0.004518,0.013629,0.012107
...,...,...,...,...,...,...
1773,2011-03-14,0.003905,-0.000407,-0.002139,-0.010953,0.004765
1774,2011-03-15,-0.008487,0.004473,-0.016724,-0.004873,0.011994
1775,2011-03-16,-0.009272,-0.017409,-0.025294,-0.020699,-0.023705
1776,2011-03-17,0.010799,-0.001236,0.027293,-0.001136,-0.005364


## Análises

### Análise de associação - Phi

- Para obter o *Phi*, precisamos obter a tabela cruzada

In [8]:
tabela = pd.crosstab(dados_tratamentos['tratamento'], 
                     dados_tratamentos['evolucao'])
tabela

evolucao,melhorou,nao-melhorou
tratamento,,
nao-tratado,26,29
tratado,35,15


- Ferramenta scipy.stats

In [9]:
# obter valor do qui quadrado
qui_quadrado = st.chi2_contingency(tabela)[0]
qui_quadrado

4.6625668947297125

In [10]:
# tamanho da amostra
n = np.sum(np.sum(tabela))
n

105

In [11]:
# obter o phi
Phi = np.sqrt(qui_quadrado / n)
Phi

0.21072588592155886

- Teste qui-quadrado

In [12]:
qui, p, gl, t_esp = st.chi2_contingency(tabela)

In [13]:
qui, p

(4.6625668947297125, 0.030827072412198585)

### Análise de associação - *V* de *Cramer*

- Filtrar respostas incompletas

In [14]:
dados_nps_filtrados = dados_nps[(dados_nps['response_status'] == 'Complete') & \
                                (dados_nps['nps_score'].notna()) & \
                                (dados_nps['gender'].notna())].reset_index()

- Criar grupos do NPS

In [15]:
def create_nps_groups(x):

  if x <= 5:
    return 'detratores'
  elif x >= 9:
    return 'promotores'
  else:
    return 'neutros'

In [16]:
dados_nps_filtrados['nps_groups'] = dados_nps_filtrados['nps_score'].apply(lambda x: create_nps_groups(x))

In [17]:
dados_nps_filtrados.groupby('nps_groups').size().to_frame('n')

,n
nps_groups,
detratores,25
neutros,171
promotores,2047


- V de Cramer: Vamos analisar a associação entre Sexo e os grupos do NPS

In [18]:
# obter a tabela cruzada
tabela = pd.crosstab(dados_nps_filtrados['gender'], dados_nps_filtrados['nps_groups'])
tabela

nps_groups,detratores,neutros,promotores
gender,,,
Female,12,105,1565
Male,13,66,482


- Método 1: utilizando a função do scipy "association"

In [19]:
vcramer_1 = st.contingency.association(tabela, method='cramer')
vcramer_1

0.1134646600588705

- Método 2: estimação direta pela formula

In [20]:
# obter valor do qui quadrado
qui_quadrado = st.chi2_contingency(tabela)[0]
qui_quadrado

28.87689583154292

In [21]:
# tamanho da amostra
n = np.sum(np.sum(tabela))
n

2243

In [22]:
# minimo entre r e c
minshape = min(tabela.shape)-1
minshape

1

In [23]:
vcramer_2 = np.sqrt(qui_quadrado / (n * minshape))
vcramer_2

0.1134646600588705

- Teste qui-quadrado

In [24]:
qui, p, gl, t_esp = st.chi2_contingency(tabela)

In [25]:
qui, p

(28.87689583154292, 5.363666207418583e-07)

### Análise de Correlação

- Verificar nulos

In [26]:
dados_bolsa[dados_bolsa.isnull()]

,data,petr4,bbdc3,vale5,ambv4,itub4
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1773,NaN,NaN,NaN,NaN,NaN,NaN
1774,NaN,NaN,NaN,NaN,NaN,NaN
1775,NaN,NaN,NaN,NaN,NaN,NaN
1776,NaN,NaN,NaN,NaN,NaN,NaN


- Tratar os nulos

In [27]:
dados_bolsa_filtrados = dados_bolsa.dropna()

In [31]:
dados_bolsa_filtrados.agg(min_data = pd.NamedAgg('data', 'min'),
                          max_data = pd.NamedAgg('data', 'max'))

,data
min_data,2004-05-26
max_data,2011-03-18


#### Correlação de Pearson

In [ ]:
# correlacao
pearson_correl = dados_bolsa_filtrados.corr()
pearson_correl

,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.000000,0.539247,0.724023,0.392074,0.593834
bbdc3,0.539247,1.000000,0.592143,0.470529,0.778506
vale5,0.724023,0.592143,1.000000,0.482919,0.642838
ambv4,0.392074,0.470529,0.482919,1.000000,0.488886
itub4,0.593834,0.778506,0.642838,0.488886,1.000000


In [ ]:
# matriz ajustada
pearson_correl.style.background_gradient(cmap='coolwarm').set_precision(2)

,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.00,0.54,0.72,0.39,0.59
bbdc3,0.54,1.00,0.59,0.47,0.78
vale5,0.72,0.59,1.00,0.48,0.64
ambv4,0.39,0.47,0.48,1.00,0.49
itub4,0.59,0.78,0.64,0.49,1.00


- Verificar se correlações são significantes

In [ ]:
# colunas para correlacionar
correl_columns = ['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']

In [ ]:
# teste
pg.pairwise_corr(dados_bolsa_filtrados, 
                 columns = correl_columns, 
                 method = 'pearson')

,X,Y,method,alternative,n,r,CI95%,p-unc,BF10,power
0,petr4,bbdc3,pearson,two-sided,1604,0.539247,"[0.5, 0.57]",1.094616e-121,7.488e+117,1.0
1,petr4,vale5,pearson,two-sided,1604,0.724023,"[0.7, 0.75]",1.122582e-260,3.654e+256,1.0
2,petr4,ambv4,pearson,two-sided,1604,0.392074,"[0.35, 0.43]",4.390573e-60,3.057e+56,1.0
3,petr4,itub4,pearson,two-sided,1604,0.593834,"[0.56, 0.62]",1.793002e-153,3.791e+149,1.0
4,bbdc3,vale5,pearson,two-sided,1604,0.592143,"[0.56, 0.62]",2.143940e-152,3.189e+148,1.0
5,bbdc3,ambv4,pearson,two-sided,1604,0.470529,"[0.43, 0.51]",3.714265e-89,2.774e+85,1.0
6,bbdc3,itub4,pearson,two-sided,1604,0.778506,"[0.76, 0.8]",0.000000e+00,inf,1.0
7,vale5,ambv4,pearson,two-sided,1604,0.482919,"[0.44, 0.52]",1.738600e-94,5.688e+90,1.0
8,vale5,itub4,pearson,two-sided,1604,0.642838,"[0.61, 0.67]",1.064830e-187,5.347e+183,1.0
9,ambv4,itub4,pearson,two-sided,1604,0.488886,"[0.45, 0.53]",3.929767e-97,2.467e+93,1.0


#### Correlação de Spearman

In [ ]:
# correlacao
spearman_correl = dados_bolsa_filtrados.corr(method='spearman')
spearman_correl

,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.000000,0.472143,0.644211,0.330002,0.531198
bbdc3,0.472143,1.000000,0.505002,0.419052,0.758946
vale5,0.644211,0.505002,1.000000,0.404373,0.574199
ambv4,0.330002,0.419052,0.404373,1.000000,0.449118
itub4,0.531198,0.758946,0.574199,0.449118,1.000000


In [ ]:
# matriz ajustada
spearman_correl.style.background_gradient(cmap='coolwarm').set_precision(2)

,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.00,0.47,0.64,0.33,0.53
bbdc3,0.47,1.00,0.51,0.42,0.76
vale5,0.64,0.51,1.00,0.40,0.57
ambv4,0.33,0.42,0.40,1.00,0.45
itub4,0.53,0.76,0.57,0.45,1.00


- Teste de significância

In [ ]:
# colunas para correlacionar
correl_columns = ['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']

In [ ]:
# teste
pg.pairwise_corr(dados_bolsa_filtrados, 
                 columns = correl_columns, 
                 method = 'spearman')

,X,Y,method,alternative,n,r,CI95%,p-unc,power
0,petr4,bbdc3,spearman,two-sided,1604,0.472143,"[0.43, 0.51]",7.723804e-90,1.0
1,petr4,vale5,spearman,two-sided,1604,0.644211,"[0.61, 0.67]",9.494683e-189,1.0
2,petr4,ambv4,spearman,two-sided,1604,0.330002,"[0.29, 0.37]",4.669599e-42,1.0
3,petr4,itub4,spearman,two-sided,1604,0.531198,"[0.5, 0.57]",1.762927e-117,1.0
4,bbdc3,vale5,spearman,two-sided,1604,0.505002,"[0.47, 0.54]",1.513500e-104,1.0
5,bbdc3,ambv4,spearman,two-sided,1604,0.419052,"[0.38, 0.46]",3.166567e-69,1.0
6,bbdc3,itub4,spearman,two-sided,1604,0.758946,"[0.74, 0.78]",8.700692e-301,1.0
7,vale5,ambv4,spearman,two-sided,1604,0.404373,"[0.36, 0.44]",3.794800e-64,1.0
8,vale5,itub4,spearman,two-sided,1604,0.574199,"[0.54, 0.61]",2.395419e-141,1.0
9,ambv4,itub4,spearman,two-sided,1604,0.449118,"[0.41, 0.49]",1.896860e-80,1.0


#### Correlação de Kendall

In [ ]:
# correlacao
kendall_correl = dados_bolsa_filtrados.corr(method='kendall')
kendall_correl

,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.000000,0.332691,0.471033,0.231311,0.378755
bbdc3,0.332691,1.000000,0.358557,0.294492,0.571953
vale5,0.471033,0.358557,1.000000,0.282942,0.411663
ambv4,0.231311,0.294492,0.282942,1.000000,0.318638
itub4,0.378755,0.571953,0.411663,0.318638,1.000000


In [ ]:
# matriz ajustada
kendall_correl.style.background_gradient(cmap='coolwarm').set_precision(2)

,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.00,0.33,0.47,0.23,0.38
bbdc3,0.33,1.00,0.36,0.29,0.57
vale5,0.47,0.36,1.00,0.28,0.41
ambv4,0.23,0.29,0.28,1.00,0.32
itub4,0.38,0.57,0.41,0.32,1.00


- Teste de significância

In [ ]:
# colunas para correlacionar
correl_columns = ['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']

In [ ]:
# teste
pg.pairwise_corr(dados_bolsa_filtrados, 
                 columns = correl_columns, 
                 method = 'kendall')

,X,Y,method,alternative,n,r,CI95%,p-unc,power
0,petr4,bbdc3,kendall,two-sided,1604,0.332691,"[0.29, 0.38]",1.358166e-88,1.0
1,petr4,vale5,kendall,two-sided,1604,0.471033,"[0.43, 0.51]",9.805061e-176,1.0
2,petr4,ambv4,kendall,two-sided,1604,0.231311,"[0.18, 0.28]",8.582460e-44,1.0
3,petr4,itub4,kendall,two-sided,1604,0.378755,"[0.34, 0.42]",2.531465e-114,1.0
4,bbdc3,vale5,kendall,two-sided,1604,0.358557,"[0.32, 0.4]",1.375615e-102,1.0
5,bbdc3,ambv4,kendall,two-sided,1604,0.294492,"[0.25, 0.34]",8.231269e-70,1.0
6,bbdc3,itub4,kendall,two-sided,1604,0.571953,"[0.54, 0.6]",7.030155e-258,1.0
7,vale5,ambv4,kendall,two-sided,1604,0.282942,"[0.24, 0.33]",1.246269e-64,1.0
8,vale5,itub4,kendall,two-sided,1604,0.411663,"[0.37, 0.45]",1.100820e-134,1.0
9,ambv4,itub4,kendall,two-sided,1604,0.318638,"[0.27, 0.36]",1.852468e-81,1.0
